# Satellite Configuration

[Satellites](../api_reference/sats/index.rst) are the basic unit of agent in the 
environment. Four things must be specified in subclasses of `Satellite`:

* The `observation_spec`, which defines the satellite's [observation](../api_reference/obs/index.rst).
* The `action_spec`, which defines the satellite's [actions](../api_reference/act/index.rst).
* The `dyn_type`,  which selects the underlying [dynamics model](../api_reference/sim/dyn.rst) used in simulation.
* The `fsw_type`,  which selects the underlying [flight software model](../api_reference/sim/fsw.rst).

A very simple satellite is defined below:

In [1]:
from bsk_rl import sats, act, obs, scene, data, SatelliteTasking
from bsk_rl.sim import dyn, fsw
import numpy as np

from Basilisk.architecture import bskLogging
bskLogging.setDefaultLogLevel(bskLogging.BSK_WARNING)


class SimpleSatellite(sats.Satellite):
    observation_spec = [obs.Time()]  # Passed as list of instantiated classes
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel  # Passed as a type
    fsw_type = fsw.BasicFSWModel

## Setting Satellite Parameters

Without instantiating the satellite, parameters that can be set in the various models
can be inspected.

In [2]:
SimpleSatellite.default_sat_args()

{'hs_min': 0.0,
 'maxCounterValue': 4,
 'thrMinFireTime': 0.02,
 'desatAttitude': 'sun',
 'controlAxes_B': [1, 0, 0, 0, 1, 0, 0, 0, 1],
 'thrForceSign': 1,
 'K': 7.0,
 'Ki': -1,
 'P': 35.0,
 'utc_init': 'this value will be set by the world model',
 'batteryStorageCapacity': 288000.0,
 'storedCharge_Init': <function bsk_rl.sim.dyn.BasicDynamicsModel.<lambda>()>,
 'disturbance_vector': None,
 'dragCoeff': 2.2,
 'basePowerDraw': 0.0,
 'wheelSpeeds': <function bsk_rl.sim.dyn.BasicDynamicsModel.<lambda>()>,
 'maxWheelSpeed': inf,
 'u_max': 0.2,
 'rwBasePower': 0.4,
 'rwMechToElecEfficiency': 0.0,
 'rwElecToMechEfficiency': 0.5,
 'panelArea': 1.0,
 'panelEfficiency': 0.2,
 'nHat_B': array([ 0,  0, -1]),
 'mass': 330,
 'width': 1.38,
 'depth': 1.04,
 'height': 1.58,
 'sigma_init': <function bsk_rl.sim.dyn.BasicDynamicsModel.<lambda>()>,
 'omega_init': <function bsk_rl.sim.dyn.BasicDynamicsModel.<lambda>()>,
 'rN': None,
 'vN': None,
 'oe': <function bsk_rl.utils.orbital.random_orbit(i: Option

These parameters can be overriden when instantiating the satellite through the `sat_args`
argument. 

In [3]:
sat = SimpleSatellite(
    name="SimpleSat-1",
    sat_args=dict(
        mass=300,  # Setting a constant value
        dragCoeff=lambda: np.random.uniform(2.0, 2.4),  # Setting a randomized value
    ),
)


Each time the simulation is reset, all of the function-based randomizers are called.

In [4]:
sat.generate_sat_args()  # Called by the environment on reset()
sat.sat_args

{'hs_min': 0.0,
 'maxCounterValue': 4,
 'thrMinFireTime': 0.02,
 'desatAttitude': 'sun',
 'controlAxes_B': [1, 0, 0, 0, 1, 0, 0, 0, 1],
 'thrForceSign': 1,
 'K': 7.0,
 'Ki': -1,
 'P': 35.0,
 'utc_init': 'this value will be set by the world model',
 'batteryStorageCapacity': 288000.0,
 'storedCharge_Init': 248147.2315151184,
 'disturbance_vector': None,
 'dragCoeff': 2.061240388324582,
 'basePowerDraw': 0.0,
 'wheelSpeeds': array([1438.84452174,  539.41721298,  414.34417058]),
 'maxWheelSpeed': inf,
 'u_max': 0.2,
 'rwBasePower': 0.4,
 'rwMechToElecEfficiency': 0.0,
 'rwElecToMechEfficiency': 0.5,
 'panelArea': 1.0,
 'panelEfficiency': 0.2,
 'nHat_B': array([ 0,  0, -1]),
 'mass': 300,
 'width': 1.38,
 'depth': 1.04,
 'height': 1.58,
 'sigma_init': array([0.75998917, 0.68479778, 0.44475858]),
 'omega_init': array([ 8.81683082e-05, -3.40923770e-05,  5.66447569e-05]),
 'rN': None,
 'vN': None,
 'oe': <Basilisk.utilities.orbitalMotion.ClassicElements at 0x2b0757d60>,
 'mu': 398600436000000

As a result, each episode will have different randomized parameters:

In [5]:
for _ in range(3):
    sat.generate_sat_args()  # Called by the environment on reset()
    print("New value of dragCoeff:", sat.sat_args["dragCoeff"])

New value of dragCoeff: 2.075029823723969
New value of dragCoeff: 2.2510606287669623
New value of dragCoeff: 2.1255723715820993


## The Observation Specification

A variety of observation elements are available for satellites. Full documentation
can be [found here](../api_reference/obs/index.rst), but some commonly used elements
are explored below.

<div class="alert alert-info">

**Info:** In these examples, `obs_type=dict` is passed to the `Satellite` constructor
so that the observation is human readable. While some RL libraries support dictionary-based
observations, the default return type - the numpy array format - is more typically used.

</div>


### Satellite Properties

The most common type of observations is introspective; i.e. what is my current state?
Any `@property` in the `dyn_type` or `fsw_type` of the satellite can be accessed using
SatProperties.

In [6]:
class SatPropsSatellite(sats.Satellite):
    observation_spec = [
        obs.SatProperties(
            # At a minimum, specify the property to observe
            dict(prop="wheel_speeds"),
            # You can specify the module to use for the observation, but it is not necessary
            # if only one module has for the property
            dict(prop="battery_charge_fraction", module="dynamics"), 
            # Properties can be normalized by some constant. This is generally desirable
            # for RL algorithms to keep values around [-1, 1].
            dict(prop="r_BN_P", norm=7e6),
        )
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=SatPropsSatellite("PropSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

{'sat_props': {'wheel_speeds': array([-69.65366005,  82.90689294, -76.49213264]),
  'battery_charge_fraction': 0.48649422083945004,
  'r_BN_P_normd': array([ 0.93736257,  0.01856028, -0.2906706 ])}}

In some cases, you may want to access a bespoke property that is not natively implemented
in a model. To do that, simply extend the model with your desired property.

In [7]:
class BespokeFSWModel(fsw.BasicFSWModel):
    @property
    def meaning_of_life(self):
        return 42
    
class BespokeSatPropsSatellite(sats.Satellite):
    observation_spec = [
        obs.SatProperties(dict(prop="meaning_of_life"))
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = BespokeFSWModel

env = SatelliteTasking(
    satellite=BespokeSatPropsSatellite("BespokeSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

{'sat_props': {'meaning_of_life': 42.0}}

Alternatively, define the property with a function that takes the satellite object as an argument.

In [8]:
class CustomSatPropsSatellite(sats.Satellite):
    observation_spec = [
        obs.SatProperties(dict(prop="meaning_of_life", fn=lambda sat: 42))
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=CustomSatPropsSatellite("BespokeSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

{'sat_props': {'meaning_of_life': 42.0}}

### Opportunity Properties
Another common input to the observation is information about upcoming locations that 
are being accessed by the satellite. Currently, these include ground stations for
downlink and targets for imaging, but `OpportunityProperties` will work with any
location added by `add_location_for_access_checking`. In these examples, 

In [9]:
class OppPropsSatellite(sats.ImagingSatellite):
    observation_spec = [
        obs.OpportunityProperties(
            # Properties can be added by some default names
            dict(prop="priority"), 
            # They can also be normalized
            dict(prop="opportunity_open", norm=5700.0),
            # Or they can be specified by an arbitrary function
            dict(fn=lambda sat, opp: opp["r_LP_P"] + 42),
            n_ahead_observe=3,
        )
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.ImagingDynModel
    fsw_type = fsw.ImagingFSWModel

env = SatelliteTasking(
    satellite=OppPropsSatellite("OppSat-1", {}, obs_type=dict),
    scenario=scene.UniformTargets(1000),
    rewarder=data.UniqueImageReward(),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

{'target': {'target_0': {'priority': 0.4443951646172174,
   'opportunity_open_normd': 0.0015758551567928273,
   'prop_2': array([-5343878.29888511,  -788528.52510135,  3391398.49178388])},
  'target_1': {'priority': 0.573932415809538,
   'opportunity_open_normd': 0.025210644018583337,
   'prop_2': array([-4713211.84095004,  -993973.24625346,  4180687.69286191])},
  'target_2': {'priority': 0.04978591822163059,
   'opportunity_open_normd': 0.02440022571056314,
   'prop_2': array([-5029539.89292592, -1451360.77285074,  3643853.52937037])}}}


### Navigating the Observation

Usually, multiple observation types need to be composed to sufficiently represent the
environment for the learning agent. Simply add multiple observations to the observation
specification list to combine them in the observation.


In [10]:
class ComposedObsSatellite(sats.Satellite):
    observation_spec = [
        obs.Eclipse(),
        obs.SatProperties(dict(prop="battery_charge_fraction"))
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=ComposedObsSatellite("PropSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

{'eclipse': [720.0, 2730.0],
 'sat_props': {'battery_charge_fraction': 0.8430167689326733}}


A few useful functions exist for inspecting the observation. The `observation_space`
property of the satellite and the environment return a Gym observation space to describe
the observation. In the single agent `SatelliteTasking` environment, these are the same.

<div class="alert alert-info">

**Info:** Here, we return to the `ndarray` default observation type.

</div>

In [11]:
env = SatelliteTasking(
    satellite=ComposedObsSatellite("PropSat-1", {}),
    log_level="CRITICAL",
)
(env.observation_space, env.unwrapped.satellite.observation_space)

(Box(-1e+16, 1e+16, (3,), float64), Box(-1e+16, 1e+16, (3,), float64))


With the flattened-vector type observation, it can be hard for the user to relate
elements to specific observations.


In [12]:
observation, _ = env.reset()
observation

array([4.7700000e+03, 1.1100000e+03, 7.2316086e-01])

The `observation_description` property can help the user understand what elements are 
present in the observation.

In [13]:
env.unwrapped.satellite.observation_description

['eclipse[0]', 'eclipse[1]', 'sat_props.battery_charge_fraction']


## The Action Specification

The [action specification](../api_reference/act/index.rst) works similarly to observation
specification. A list of actions is set in the class definition of the satellite.

In [14]:
class ActionSatellite(sats.Satellite):
    observation_spec = [obs.Time()]
    action_spec = [
        # If action duration is not set, the environment max_step_duration will be used;
        # however, being explicit is always preferable
        act.Charge(duration=120.0),
        act.Desat(duration=60.0),
        # One action can be included multiple time, if different settings are desired
        act.Charge(duration=600.0,),
    ]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=ActionSatellite("ActSat-1", {}, obs_type=dict),
    log_level="INFO",
)
env.reset()

# Try each action; index corresponds to the order of addition
_ =env.step(0)
_ =env.step(1)
_ =env.step(2)

2024-09-11 09:27:20,814                                WARNING    Creating logger for new env on PID=63128. Old environments in process may now log times incorrectly.


2024-09-11 09:27:21,055 gym                            INFO       Resetting environment with seed=522560704


2024-09-11 09:27:21,124 gym                            INFO       <0.00> Environment reset


2024-09-11 09:27:21,124 gym                            INFO       <0.00> === STARTING STEP ===


2024-09-11 09:27:21,124 sats.satellite.ActSat-1        INFO       <0.00> ActSat-1: action_charge tasked for 120.0 seconds


...

2024-09-11 09:27:21,138 sats.satellite.ActSat-1        INFO       <180.00> ActSat-1: setting timed terminal event at 780.0


2024-09-11 09:27:21,170 sats.satellite.ActSat-1        INFO       <780.00> ActSat-1: timed termination at 780.0 for action_charge


2024-09-11 09:27:21,170 data.base                      INFO       <780.00> Data reward: {'ActSat-1': 0.0}


2024-09-11 09:27:21,170 sats.satellite.ActSat-1        INFO       <780.00> ActSat-1: Satellite ActSat-1 requires retasking


2024-09-11 09:27:21,171 gym                            INFO       <780.00> Step reward: 0.0


As with the observations, properties exist to help understand the actions available.

In [15]:
env.action_space

Discrete(3)

In [16]:
env.unwrapped.satellite.action_description

['action_charge', 'action_desat', 'action_charge']

Some actions take additional configurations, add multiple actions to the satellite, and/or
have "special" features that are useful for manually interacting with the environment. 
For example, the imaging action can add an arbitrary number of actions corresponding to
upcoming targets and process the name of a target directly instead of operating by
action index.

In [17]:
class ImageActSatellite(sats.ImagingSatellite):
    observation_spec = [obs.Time()]
    action_spec = [
        # Set the number of upcoming targets to consider
        act.Image(n_ahead_image=3)
    ]
    dyn_type = dyn.ImagingDynModel
    fsw_type = fsw.ImagingFSWModel

env = SatelliteTasking(
    satellite=ImageActSatellite("ActSat-2", {}),
    scenario=scene.UniformTargets(1000),
    rewarder=data.UniqueImageReward(),
    log_level="INFO",
)
env.reset()

env.unwrapped.satellite.action_description

2024-09-11 09:27:21,181                                WARNING    Creating logger for new env on PID=63128. Old environments in process may now log times incorrectly.


2024-09-11 09:27:21,484 gym                            INFO       Resetting environment with seed=950797704


2024-09-11 09:27:21,484 scene.targets                  INFO       Generating 1000 targets


2024-09-11 09:27:22,163 gym                            INFO       <0.00> Environment reset


['action_image_0', 'action_image_1', 'action_image_2']

Demonstrating the action overload feature, we task the satellite based on target name.
While this is not part of the official Gym API, we find it useful in certain cases.

In [18]:
target = env.unwrapped.satellite.find_next_opportunities(n=10)[9]["object"]
_ = env.step(target)

2024-09-11 09:27:22,168 sats.satellite.ActSat-2        INFO       <0.00> ActSat-2: Finding opportunity windows from 0.00 to 600.00 seconds


2024-09-11 09:27:22,186 sats.satellite.ActSat-2        INFO       <0.00> ActSat-2: Finding opportunity windows from 600.00 to 1200.00 seconds


2024-09-11 09:27:22,204 sats.satellite.ActSat-2        INFO       <0.00> ActSat-2: Finding opportunity windows from 1200.00 to 1800.00 seconds


2024-09-11 09:27:22,223 gym                            INFO       <0.00> === STARTING STEP ===


2024-09-11 09:27:22,223 act.discrete_actions           WARNING    <0.00> Action 'Target(tgt-822)' is not an integer. Will attempt to use compatible set_action_override method.


...

2024-09-11 09:27:22,224 sats.satellite.ActSat-2        INFO       <0.00> ActSat-2: setting timed terminal event at 1328.7


2024-09-11 09:27:22,321 sats.satellite.ActSat-2        INFO       <1329.00> ActSat-2: timed termination at 1328.7 for Target(tgt-822) window


2024-09-11 09:27:22,322 data.base                      INFO       <1329.00> Data reward: {'ActSat-2': 0.0}


2024-09-11 09:27:22,323 sats.satellite.ActSat-2        INFO       <1329.00> ActSat-2: Satellite ActSat-2 requires retasking


2024-09-11 09:27:22,323 gym                            INFO       <1329.00> Step reward: 0.0
